In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import time
from tqdm.notebook import tqdm

Сайт: https://www.kinopoisk.ru/lists/top250/
- Спарсить названия фильмов из топ-250.
- Спарсить рейтинг фильмов
- Спарсить отзывы (необязательно)

In [2]:
films = {'titles': [], 'ratings': [], 'links': []}
host = 'https://www.kinopoisk.ru'
for page in tqdm(range(1, 6)):
  url = 'https://www.kinopoisk.ru/lists/top250/?page=' + str(page) + '&tab=all'
  with requests.Session() as session:
    html = session.get(url).text
    try:
      soup = BeautifulSoup(html, 'html.parser')
      items = soup.find_all('div', class_='desktop-rating-selection-film-item')
      for item in items:
        films['titles'].append(item.find('p', class_='selection-film-item-meta__name').get_text())
        films['ratings'].append(float(item.find('span', class_='rating__value rating__value_positive').get_text()))
        films['links'].append(host + item.find('a', class_='selection-film-item-meta__link').get('href'))
    except AttributeError:
      print(url)
      continue

In [4]:
pd.DataFrame(films)

,titles,ratings,links
0,Побег из Шоушенка,8.9,https://www.kinopoisk.ru/film/326/
1,Зеленая миля,8.9,https://www.kinopoisk.ru/film/435/
2,Властелин колец: Возвращение короля,8.8,https://www.kinopoisk.ru/film/3498/
3,Властелин колец: Две крепости,8.8,https://www.kinopoisk.ru/film/312/
4,Властелин колец: Братство Кольца,8.7,https://www.kinopoisk.ru/film/328/
...,...,...,...
245,Марли и я,7.9,https://www.kinopoisk.ru/film/279850/
246,Гангстер,7.9,https://www.kinopoisk.ru/film/252156/
247,Гарри Поттер и Кубок огня,7.9,https://www.kinopoisk.ru/film/8408/
248,Холодное сердце,7.9,https://www.kinopoisk.ru/film/493208/


In [15]:
films['reviews'] = []
for link in tqdm(films['links']):
  with requests.Session() as session:
    html = session.get(link).text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='styles_filmSynopsis__zLClu')
    for item in items:
      films['reviews'].append(soup.find('p', class_='styles_paragraph__2Otvx').get_text())